In [113]:
import pandas as pd
from datetime import datetime, timedelta
import re # For regular expressions to parse filename

# --- Configuration ---
ASSUMED_YEAR = 2024 # General assumption, but file2 will try to use its own year
ASSUMED_MONTH = 10
ASSUMED_DAY = 3
MATCHING_WINDOW_SECONDS = 120
# NEW: Configuration for df1 datetime adjustment
FILE1_DATETIME_SUBTRACT_HOURS = 4 # Hours to subtract from parsed df1 datetime

# --- Helper Functions ---
def parse_test1_filename_to_datetime(filename_str, year, month, day):
    # (Keep this function as it was, it returns the raw parsed datetime)
    match = re.search(r'cap-(\d{2}-\d{2}-\d{2}\.\d{3})\.jpg', filename_str, re.IGNORECASE)
    if not match:
        match_alt = re.search(r'(\d{2})[-_:]?(\d{2})[-_:]?(\d{2})[\._](\d{3})', filename_str)
        if match_alt:
            h, m, s, ms_str = match_alt.groups()
            try:
                return datetime(
                    year, month, day,
                    int(h), int(m), int(s), int(ms_str) * 1000
                )
            except ValueError as e:
                # print(f"Error parsing time from filename (alt pattern) '{filename_str}': {e}")
                return None
        else:
            # print(f"Filename '{filename_str}' did not match expected patterns for time extraction.")
            return None

    time_part_str = match.group(1)
    try:
        h, m, s_ms = time_part_str.split('-')
        s, ms = s_ms.split('.')
        return datetime(
            year, month, day,
            int(h), int(m), int(s), int(ms) * 1000
        )
    except ValueError as e:
        # print(f"Error parsing time from filename '{filename_str}': {e}")
        return None

def parse_test2_datetime(date_str, time_str):
    """
    Parses date (e.g., "2024/10/03" or "15-Sep") and time (e.g., "07:17:11") to a datetime object.
    """
    date_str = str(date_str)
    time_str = str(time_str)
    full_datetime_str = f"{date_str} {time_str}"
    try:
        return datetime.strptime(full_datetime_str, "%Y/%m/%d %H:%M:%S")
    except ValueError:
        pass
    try:
        return datetime.strptime(full_datetime_str, "%d-%b-%Y %H:%M:%S")
    except ValueError:
        pass
    if " " in date_str and ":" in date_str:
        try:
            return datetime.strptime(date_str, "%Y/%m/%d %H:%M:%S")
        except ValueError:
            pass
        try:
            return datetime.strptime(date_str, "%d-%b-%Y %H:%M:%S")
        except ValueError:
            pass
    print(f"Error: Could not parse date/time from strings: Date='{date_str}', Time='{time_str}' with known formats.")
    return None


# --- Main Script ---
def main():
    # --- Configuration for File Paths ---
    file1_path = r"C:\Users\olivi\Desktop\PhaseOne IXM-100 photos\.csv files\oct_3_photos_in_flightlines.csv"
    file1_skiprows = 6
    file1_assumed_year = 2024
    file1_assumed_month = 10
    file1_assumed_day = 3
    file1_processed_output_path = "oct_3_photos_with_parsed_datetime_adjusted.csv" # Adjusted name

    file2_path = r"\\192.168.3.33\nextcloud_2024\Aurora 2025\Deliverables\Radiometric Data\oct_3_\assay_3478.csv"

    # 1. Load df1
    print(f"Loading df1 from: {file1_path}")
    try:
        df1 = pd.read_csv(file1_path, skiprows=file1_skiprows)
    except FileNotFoundError:
        print(f"Error: File not found at {file1_path}")
        return
    except Exception as e:
        print(f"Error loading {file1_path}: {e}")
        return

    df1.columns = df1.columns.str.strip()
    filename_col_df1 = '# Columns: Filename'
    latitude_col_df1 = 'Vehicle Latitude'
    longitude_col_df1 = 'Vehicle Longitude'
    altitude_col_df1 = 'Vehicle Altitude'

    # Basic column existence checks
    required_cols_df1 = [filename_col_df1, latitude_col_df1, longitude_col_df1, altitude_col_df1]
    for col in required_cols_df1:
        if col not in df1.columns:
            print(f"Error: Column '{col}' not found in {file1_path}. Cols: {df1.columns.tolist()}")
            return

    # Define the time adjustment for df1
    df1_time_adjustment = timedelta(hours=FILE1_DATETIME_SUBTRACT_HOURS)

    df1_processed = pd.DataFrame({
        'original_filename': df1[filename_col_df1],
        'datetime': df1[filename_col_df1].apply(
            lambda x: (dt_obj - df1_time_adjustment)  # Apply adjustment here
                      if (dt_obj := parse_test1_filename_to_datetime(x, file1_assumed_year, file1_assumed_month, file1_assumed_day)) is not None
                      else None
        ),
        'latitude': df1[latitude_col_df1],
        'longitude': df1[longitude_col_df1],
        'altitude': df1[altitude_col_df1]
    })
    df1_processed.dropna(subset=['datetime'], inplace=True)

    if df1_processed.empty:
        print(f"No valid records with parseable datetimes found in {file1_path} after processing.")
    else:
        print(f"Loaded and processed {len(df1_processed)} valid records from {file1_path} (datetime adjusted by -{FILE1_DATETIME_SUBTRACT_HOURS} hours).")
        try:
            df1_processed.to_csv(file1_processed_output_path, index=False)
            print(f"Saved processed df1 with adjusted datetimes to: {file1_processed_output_path}")
        except Exception as e:
            print(f"Error saving processed df1 to {file1_processed_output_path}: {e}")
        if df1_processed.empty:
            return # Stop if no data to match against

    # 2. Load df2
    print(f"\nLoading df2 from: {file2_path}")
    try:
        df2 = pd.read_csv(file2_path)
    except FileNotFoundError:
        print(f"Error: File not found at {file2_path}")
        return
    except Exception as e:
        print(f"Error loading {file2_path}: {e}")
        return

    df2.columns = df2.columns.str.strip()
    date_col_df2 = 'Date'
    time_col_df2 = 'Time'
    id_col_df2 = 'Id'
    latitude_col_df2 = 'Latitude'
    longitude_col_df2 = 'Longitude'
    altitude_col_df2 = 'Altitude' # Target column name

    required_cols_df2 = [date_col_df2, time_col_df2] # Id is optional for processing logic
    for col in required_cols_df2:
        if col not in df2.columns:
            print(f"Error: Column '{col}' not found in {file2_path}. Cols: {df2.columns.tolist()}")
            return

    df2['datetime'] = df2.apply(lambda row: parse_test2_datetime(row[date_col_df2], row[time_col_df2]), axis=1)
    df2_valid_datetime_mask = df2['datetime'].notna()
    df2_processed = df2[df2_valid_datetime_mask].copy()

    if len(df2_processed) < len(df2):
        print(f"Warning: {len(df2) - len(df2_processed)} rows from {file2_path} had invalid date/time and were skipped.")
    if df2_processed.empty:
        print(f"No valid records with parseable datetimes found in {file2_path} after processing. Check date/time formats.")
        return
    print(f"Loaded {len(df2_processed)} valid records from {file2_path} for processing.")

    # 3. Match and Update
    cols_to_populate = {
        latitude_col_df2: 0.0,
        longitude_col_df2: 0.0,
        altitude_col_df2: 0.0,
        'Time_Diff_To_Match_Sec': pd.NA
    }

    for col, default_val in cols_to_populate.items():
        if col not in df2_processed.columns:
            df2_processed[col] = default_val
        # Ensure numeric for lat, lon, alt if they exist but might be non-numeric
        if col not in ['Time_Diff_To_Match_Sec']:
             df2_processed[col] = pd.to_numeric(df2_processed[col], errors='coerce').fillna(default_val)


    matches_found = 0
    for index2, row2 in df2_processed.iterrows():
        time2 = row2['datetime']
        if pd.isna(time2): continue

        best_match_info = None
        min_time_diff_seconds = MATCHING_WINDOW_SECONDS + 1

        for _, row1 in df1_processed.iterrows():
            time1 = row1['datetime']
            # if pd.isna(time1): continue # Should not happen due to dropna on df1_processed

            time_diff = abs(time1 - time2)
            current_diff_seconds = time_diff.total_seconds()

            if current_diff_seconds <= MATCHING_WINDOW_SECONDS:
                if current_diff_seconds < min_time_diff_seconds:
                    min_time_diff_seconds = current_diff_seconds
                    best_match_info = {
                        'latitude': row1['latitude'],
                        'longitude': row1['longitude'],
                        'altitude': row1['altitude'],
                        'time_diff_seconds': current_diff_seconds
                    }

        if best_match_info:
            df2_processed.loc[index2, latitude_col_df2] = best_match_info['latitude']
            df2_processed.loc[index2, longitude_col_df2] = best_match_info['longitude']
            df2_processed.loc[index2, altitude_col_df2] = best_match_info['altitude']
            df2_processed.loc[index2, 'Time_Diff_To_Match_Sec'] = best_match_info['time_diff_seconds']
            matches_found += 1

    print(f"\nProcessed {len(df2_processed)} records from df2. Found {matches_found} matches within ±{MATCHING_WINDOW_SECONDS} seconds.")

    # Update the original df2 with the processed rows
    # Ensure columns to be updated exist in original df2, adding them if necessary with default values for non-matched rows
    for col_to_update in [latitude_col_df2, longitude_col_df2, altitude_col_df2, 'Time_Diff_To_Match_Sec']:
        if col_to_update not in df2.columns:
            if col_to_update == 'Time_Diff_To_Match_Sec':
                df2[col_to_update] = pd.NA
            else: # lat, lon, alt
                df2[col_to_update] = 0.0 # Default for rows in df2 not in df2_processed

    # Update df2 with values from df2_processed.
    # This transfers matched lat/lon/alt and Time_Diff_To_Match_Sec.
    # Rows in df2 that were filtered out (not in df2_processed) will retain their original values
    # or the defaults set in the loop above if the columns were newly added to df2.
    update_cols = [latitude_col_df2, longitude_col_df2, altitude_col_df2, 'Time_Diff_To_Match_Sec']
    df2.update(df2_processed[update_cols])


    # 4. Save the Result
    output_filename = 'oct_3_assay_updated_with_latlonalt_v2.csv' # Adjusted output name

    cols_to_drop_from_output = ['datetime']
    df2_output = df2.copy()
    for col in cols_to_drop_from_output:
        if col in df2_output.columns:
            df2_output = df2_output.drop(columns=[col])

    try:
        df2_output.to_csv(output_filename, index=False)
        print(f"\nUpdated data for df2 saved to {output_filename}")
    except Exception as e:
        print(f"Error saving output file for df2: {e}")

if __name__ == '__main__':
    main()

Loading df1 from: C:\Users\olivi\Desktop\PhaseOne IXM-100 photos\.csv files\oct_3_photos_in_flightlines.csv
Loaded and processed 4277 valid records from C:\Users\olivi\Desktop\PhaseOne IXM-100 photos\.csv files\oct_3_photos_in_flightlines.csv (datetime adjusted by -4 hours).
Saved processed df1 with adjusted datetimes to: oct_3_photos_with_parsed_datetime_adjusted.csv

Loading df2 from: \\192.168.3.33\nextcloud_2024\Aurora 2025\Deliverables\Radiometric Data\oct_3_\assay_3478.csv
Loaded 1784 valid records from \\192.168.3.33\nextcloud_2024\Aurora 2025\Deliverables\Radiometric Data\oct_3_\assay_3478.csv for processing.


C:\Users\olivi\AppData\Local\Temp\ipykernel_13828\299981462.py:217: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '1043.484' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df2_processed.loc[index2, altitude_col_df2] = best_match_info['altitude']



Processed 1784 records from df2. Found 1131 matches within ±120 seconds.

Updated data for df2 saved to oct_3_assay_updated_with_latlonalt_v2.csv
